# Daily Aggregations using Xarray
*(author: Grant Buster)*

Requirements:

- Install rex: `pip install NREL-rex --upgrade`
- Install dask-distributed: `pip install distributed --upgrade`
- Initialize a dask client for parallel processing
- Set dask compute chunks appropriately


In this notebook, we demonstrate how you can use ``xarray`` to quickly and easily compute daily statistics for a year of sup3rCC data. (If it seems strange to you that we're aggregating a dataset intended to improve resolution - don't worry, we think it's a bit strange too).


### Helpful tips

- Performance is really sensitive (perhaps unsurprisingly) to the dask compute chunk size you specify. The h5 chunks on disk are way too small and result in way too many operations, adding too much overhead to performance. You can get much worse performance with xarray+dask vs. a serial read when working on a small set (~1e3) locations if your compute chunks are too small. Here, we find that `(8760, 10000)` is a good compute chunk size.

- The `memory_limit` argument is the limit *per worker* in which case 100GB used here is too big even for the NREL HPC. If you are memory constrained, try setting a lower memory limit and reducing the compute chunk size.

- Setting up the full aggregate dataset and then doing one `.compute()` call resulted in chaos and memory errors (maybe too large of a memory limit!).

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from dask.distributed import Client

In [2]:
# Running on HPC node, so can use 100 processes and high memory limit
client = Client(n_workers=100, memory_limit='100GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 100
Total threads: 200,Total memory: 9.09 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40489,Workers: 100
Dashboard: http://127.0.0.1:8787/status,Total threads: 200
Started: Just now,Total memory: 9.09 TiB
Comm: tcp://127.0.0.1:41433,Total threads: 2
Dashboard: http://127.0.0.1:45137/status,Memory: 93.13 GiB
Nanny: tcp://127.0.0.1:38915,


In [3]:
%%time

year = 2015
scenario = 'ecearth3cc_ssp245_r1i1p1f1'

fp_base = '/datasets/sup3rcc/conus_{scenario}/v0.2.2_beta/sup3rcc_conus_{scenario}_{group}_{year}.h5'
fp_pr = fp_base.replace('v0.2.2_beta', 'v0.2.2_beta/daily')

kwargs = dict(engine="rex", chunks={'time_index': 8784, 'gid': 100000})
xds_trh = xr.open_mfdataset(fp_base.format(scenario=scenario, group='trh', year=year), **kwargs)
xds_wind = xr.open_mfdataset(fp_base.format(scenario=scenario, group='wind', year=year), **kwargs)
xds_pr = xr.open_mfdataset(fp_pr.format(scenario=scenario, group='pr', year=year), **kwargs)

CPU times: user 10.3 s, sys: 6.31 s, total: 16.6 s
Wall time: 59.4 s


In [4]:
%%time
da = xds_trh['temperature_2m'].groupby("time.date").max("time").astype(np.float32).compute()
da['date'] = pd.to_datetime(da['date'].values).astype(int)
da = da.rename({'date': 'time'})

ds_out = da.to_dataset()

CPU times: user 15.7 s, sys: 6.28 s, total: 21.9 s
Wall time: 27.9 s


In [5]:
%%time
da = xds_trh['relativehumidity_2m'].groupby("time.date").min("time").astype(np.float32).compute()
da['date'] = pd.to_datetime(da['date'].values).astype(int)
da = da.rename({'date': 'time'})
ds_out['relativehumidity_2m'] = da

CPU times: user 17 s, sys: 5.65 s, total: 22.6 s
Wall time: 27.4 s


In [6]:
%%time
da = xds_wind['windspeed_10m'].groupby("time.date").mean("time").astype(np.float32).compute()
da['date'] = pd.to_datetime(da['date'].values).astype(int)
da = da.rename({'date': 'time'})
ds_out['windspeed_10m'] = da

CPU times: user 17.3 s, sys: 5.7 s, total: 23 s
Wall time: 28.8 s


In [7]:
%%time
xds_pr['time'] = pd.to_datetime(xds_pr['time'].values - pd.Timedelta('12hr')).astype(int)
ds_out['pr'] = xds_pr['pr'].compute()

CPU times: user 2.2 s, sys: 3.16 s, total: 5.36 s
Wall time: 5.94 s


In [ ]:
%%time
ds_out['temperature_2m'].attrs['aggregation'] = 'Daily maximum'
ds_out['relativehumidity_2m'].attrs['aggregation'] = 'Daily minimum'
ds_out['windspeed_10m'].attrs['aggregation'] = 'Daily average'
ds_out['pr'].attrs['aggregation'] = 'Daily average'

encoding = {"temperature_2m": {"dtype": "int16", "scale_factor": 0.01, '_FillValue': 100, 'chunksizes': (100, 10000)},
            "windspeed_10m": {"dtype": "int16", "scale_factor": 0.01, '_FillValue': 120, 'chunksizes': (100, 10000)},
            "relativehumidity_2m": {"dtype": "uint16", "scale_factor": 0.01, '_FillValue': 101, 'chunksizes': (100, 10000)},
           }

ds_out = ds_out.drop_vars('time_index', errors='ignore')
ds_out.to_netcdf(f'sup3rcc_test_daily_{scenario}_{year}.nc', format='NETCDF4', engine="h5netcdf", encoding=encoding)

In [9]:
ds_out

<xarray.Dataset> Size: 14GB
Dimensions:              (gid: 2300000, time: 365)
Coordinates:
  * gid                  (gid) int64 18MB 0 1 2 3 ... 2299997 2299998 2299999
    latitude             (gid) float32 9MB 51.21 51.21 51.21 ... 23.17 23.17
    longitude            (gid) float32 9MB -129.0 -129.0 ... -64.38 -64.35
    elevation            (gid) int16 5MB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    timezone             (gid) float32 9MB -9.0 -9.0 -9.0 ... -4.0 -4.0 -4.0
    country              (gid) |S18 41MB b'Canada' b'Canada' ... b'nan' b'nan'
    state                (gid) |S20 46MB b'nan' b'nan' b'nan' ... b'nan' b'nan'
    county               (gid) |S33 76MB b'nan' b'nan' b'nan' ... b'nan' b'nan'
    offshore             (gid) int16 5MB 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    eez                  (gid) int16 5MB 1 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0
  * time                 (time) int64 3kB 1420070400000000000 ... 14515200000...
Data variables:
    temperature_2m       (time, gid) float32 3GB 8.02 7.97 7.91 ... 25.55 25.59
    relativehumidity_2m  (time, gid) float32 3GB 65.44 66.35 ... 72.97 73.54
    windspeed_10m        (time, gid) float32 3GB 16.8 16.98 17.21 ... 7.899 7.97
    pr                   (time, gid) float32 3GB 0.0001199 ... 1.845e-05